We want predict the areas with the most tornados. - classification 

In [1]:
# import dependencies 
import pandas as pd
import os

In [2]:
#import csv
file = os.path.join('..',"Resources", "TN_DF.csv")
df = pd.read_csv(file)
df['Fatalities'] = df['FAT']
df['Injuries'] = df['INJ']
df = df.drop(columns = ['FAT','ST','TZ','STF','DY','CLOSS','STN','OM','INJ','Shape_Leng'], axis = 1)
df

,FID,YR,MO,DATE,TIME,MAG,LOSS,SLAT,SLON,ELAT,ELON,LEN,WID,Fatalities,Injuries
0,23025,2011,4,2011-04-09,14:35:00,0,0.020,36.2971,-82.3174,36.2934,-82.3021,0.89,50,0,0
1,23027,2011,4,2011-04-09,16:00:00,1,0.075,36.3000,-82.4341,36.3000,-82.4341,3.00,150,0,0
2,23028,2011,4,2011-04-09,16:08:00,1,0.010,36.3037,-82.3923,36.3099,-82.3846,0.61,100,0,0
3,23029,2011,4,2011-04-09,16:15:00,0,0.015,36.0424,-82.5568,36.0477,-82.5393,1.04,50,0,0
4,23044,2011,4,2011-04-27,19:07:00,0,0.005,36.2255,-83.0570,36.2263,-83.0486,0.50,70,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1109,28915,1967,7,1967-07-05,14:25:00,1,4.000,35.0000,-90.0000,0.0000,0.0000,0.10,10,0,0
1110,28916,1991,11,1991-11-19,17:05:00,1,6.000,35.1300,-90.0200,35.2800,-89.9300,10.00,50,0,0
1111,28926,1992,3,1992-03-18,08:20:00,0,4.000,35.2200,-89.7700,0.0000,0.0000,1.50,50,0,0
1112,28962,1974,6,1974-06-07,11:30:00,1,4.000,35.5700,-89.6500,0.0000,0.0000,0.30,100,1,1


In [3]:
#check data types
df.dtypes

FID             int64
YR              int64
MO              int64
DATE           object
TIME           object
MAG             int64
LOSS          float64
SLAT          float64
SLON          float64
ELAT          float64
ELON          float64
LEN           float64
WID             int64
Fatalities      int64
Injuries        int64
dtype: object

In [4]:
# check for NaN values
df.isnull().values.any()

False

In [5]:
# seperate the years, so they can be further superated into bins 
df_1996 = df.loc[df['YR']>=1996]

In [6]:
# bin the after_1996 'Loss' data to match the before_1996 'Loss' data
after_1996 = df_1996['LOSS']
spending_bins = [0,.00005,.0005,.005,.05,.5,5,50,500,1000]
pd.cut(after_1996, spending_bins)
group_names = [1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0]
after_1996 = pd.cut(after_1996, spending_bins, labels=group_names)

after_1996

/Users/clarerobbins/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/io/formats/format.py:1429: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())


0       4.0
1       5.0
2       4.0
3       4.0
4       3.0
       ... 
1089    NaN
1103    4.0
1104    5.0
1107    4.0
1113    5.0
Name: LOSS, Length: 581, dtype: category
Categories (9, float64): [1.0 < 2.0 < 3.0 < 4.0 ... 6.0 < 7.0 < 8.0 < 9.0]

In [7]:
#count NaN values in after_1996
after_1996.isna().sum()

105

In [8]:
# check before_1996 values
df_1996 = df.loc[df['YR']<1996]
before_1996= df_1996['LOSS']
before_1996

14      5.0
15      4.0
16      4.0
17      3.0
18      6.0
       ... 
1108    4.0
1109    4.0
1110    6.0
1111    4.0
1112    4.0
Name: LOSS, Length: 533, dtype: float64

In [9]:
#count NaN values in before_1996
before_1996.isna().sum()

0

In [10]:
# concat before_1996 and after_1996
updated_loss_column = pd.concat([before_1996, after_1996]).sort_index()

updated_loss_column

0       4.0
1       5.0
2       4.0
3       4.0
4       3.0
       ... 
1109    4.0
1110    6.0
1111    4.0
1112    4.0
1113    5.0
Name: LOSS, Length: 1114, dtype: float64

In [11]:
#update loss column in db
df['Property_Loss'] = updated_loss_column
df= df.drop(columns='LOSS', axis=1)
df

,FID,YR,MO,DATE,TIME,MAG,SLAT,SLON,ELAT,ELON,LEN,WID,Fatalities,Injuries,Property_Loss
0,23025,2011,4,2011-04-09,14:35:00,0,36.2971,-82.3174,36.2934,-82.3021,0.89,50,0,0,4.0
1,23027,2011,4,2011-04-09,16:00:00,1,36.3000,-82.4341,36.3000,-82.4341,3.00,150,0,0,5.0
2,23028,2011,4,2011-04-09,16:08:00,1,36.3037,-82.3923,36.3099,-82.3846,0.61,100,0,0,4.0
3,23029,2011,4,2011-04-09,16:15:00,0,36.0424,-82.5568,36.0477,-82.5393,1.04,50,0,0,4.0
4,23044,2011,4,2011-04-27,19:07:00,0,36.2255,-83.0570,36.2263,-83.0486,0.50,70,0,0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1109,28915,1967,7,1967-07-05,14:25:00,1,35.0000,-90.0000,0.0000,0.0000,0.10,10,0,0,4.0
1110,28916,1991,11,1991-11-19,17:05:00,1,35.1300,-90.0200,35.2800,-89.9300,10.00,50,0,0,6.0
1111,28926,1992,3,1992-03-18,08:20:00,0,35.2200,-89.7700,0.0000,0.0000,1.50,50,0,0,4.0
1112,28962,1974,6,1974-06-07,11:30:00,1,35.5700,-89.6500,0.0000,0.0000,0.30,100,1,1,4.0


In [13]:
df.to_csv('../Resources/cleaned_tn_tornadoes.csv', index = False)